# Evaporator Example for Water Separation

This notebook describes the `Evaporator` unit model from WaterTAP and how it is used to separate seawater into fresh water and a brine solution. This is possible when using the `Evaporator` unit since it allows the use of two different property packages, one for each phase in the system. For the vapor, the property package for pure water is used while for the feed and liquid outlet (brine) the property package for seawater is used.

In this example, a given flow of seawater is fed to the `Evaporator` at given initial conditions and separated into a vapor and a liquid phase considering ideal and nonideal thermodynamic equilibrium conditions. Fresh water is separated as vapor at the top product in the evaporator, while a concentrated brine solution is separated as liquid phase at the bottom product in the evaporator unit. The seawater at the inlet is assumed to contain a fixed amount of total dissolved solids (TDS) in the water. 

This example is split into two parts. Part I documents the solution of the evaporator example assuming ideal equilibrium in the evaporator, i.e., 

$$ P = x_{\rm H_2O} \; P_{\rm H_2O}^{\rm sat} $$

Part II documents how to use and solve the evaporator model assuming nonideal equilibrium, i.e., 

$$ P = x_{\rm H_2O} \; \gamma_{\rm H_2O} \; P_{\rm H_2O}^{\rm sat} $$

while implementing the electrolyte NRTL (eNRTL) model from IDAES to calculate the activity coefficients $\gamma_i$ of solvent and solute(s). For this case study, we assume that the seawater has only a single solvent, $\rm H_2O$, and a single electrolyte is dissolved in the solvent, $\rm NaCl$, as shown in the equation below:

$$ \rm NaCl \; \; \rightarrow \; \; \rm Na^+ \; \; + \; \; \rm Cl^- $$

References:

[1] *Local Composition Model for Excess Gibbs Energy of Electrolyte Systems, Pt 1*. Chen, C.-C., Britt, H.I., Boston, J.F., Evans, L.B., AIChE Journal, 1982, Vol. 28(4), pgs. 588-596. 

[2] *Pilot demonstration of concentrated solar-poweered desalination of subsurface agricultural drainage water and other brackish groundwater sources.* Matthew D. Stuber, Christopher Sullivan, Spencer A. Kirk, Jennifer A. Farrand, Philip V. Schillai, Brian D. Fojtasek, and Aaron H. Mandell. Desalination, 355 (2015), 186-196.

## Part I: Evaporator in an Ideal System

To start the construction of the evaporator model, import all the Pyomo, IDAES, and WaterTAP libraries and packages needed in the model:

In [1]:
import logging 

# Import Pyomo libraries
import pyomo.environ as pyo
from pyomo.environ import units as pyunits
from pyomo.environ import (ConcreteModel, TransformationFactory, Block,
                           Constraint, Objective, minimize,
                           Param, value, Set, log, exp)

# Import IDAES libraries
import idaes.core.util.scaling as iscale
import idaes.logger as idaeslog
from idaes.core import FlowsheetBlock
from idaes.models.properties.modular_properties.base.generic_property import GenericParameterBlock
from idaes.core.solvers import get_solver
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import propagate_state

# Import evaporator model from WaterTap
from watertap.unit_models.mvc.components import Evaporator

logging.getLogger("pyomo.repn.plugins.nl_writer").setLevel(logging.ERROR)

Import the property packages for water and seawater to be used in the vapor and liquid outlets in the `Evaporator` unit.

In [2]:
# Import property packages from WaterTap
import watertap.property_models.seawater_prop_pack as props_sw
import watertap.property_models.water_prop_pack as props_w

After importing all libraries and packages, create a Pyomo concrete model, flowsheet, and water and seawater property parameter blocks.

In [3]:
m = ConcreteModel("Evaporator Model")

m.fs = FlowsheetBlock(dynamic=False)

m.fs.properties_vapor = props_w.WaterParameterBlock()
m.fs.properties_feed = props_sw.SeawaterParameterBlock()

Add a block to include the evaporator and variables needed to solve the evaporator under ideal thermodynamic assumptions. Inside the new block, add an `Evaporator` unit to the flowsheet with two property packages: one for the feed (liquid phase) and one for the top product (vapor phase).

In [4]:
m.fs.ideal = Block()

m.fs.ideal.evaporator = Evaporator(property_package_feed=m.fs.properties_feed,
                                   property_package_vapor=m.fs.properties_vapor)

Deactivate the equality equilibrium constraint from the `Evaporator` unit and replace it with the new equilibrium equation. That is, 
$$ P=x_{\rm H_2O} \; \gamma_{\rm H_2O} \; P_{\rm H_2O}^{\rm sat}$$

Since we are considering an ideal equilibrium, $\gamma_{\rm H_2O}$ is equal to 1 and added to the model as a Pyomo parameter `Param`.

In [5]:
m.fs.ideal.act_coeff = pyo.Param(
    initialize=1,
    units=pyunits.dimensionless,
    doc="Ideal activity coefficient for water"
)

m.fs.ideal.evaporator.eq_brine_pressure.deactivate()
@m.fs.ideal.Constraint(doc="Vapor-liquid equilibrium equation")
def _eq_ideal_phase_equilibrium(b):
    return b.evaporator.properties_brine[0].pressure == (
        m.fs.ideal.act_coeff*
        b.evaporator.properties_brine[0].mole_frac_phase_comp["Liq", "H2O"]*
        b.evaporator.properties_vapor[0].pressure_sat
    )

Add scaling factors for the properties and evaporator variables.

In [6]:
# Scaling factors for the property packages
m.fs.properties_feed.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties_feed.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))
m.fs.properties_vapor.set_default_scaling("flow_mass_phase_comp", 1, index=("Vap", "H2O"))
m.fs.properties_vapor.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))

# Scaling factors for the evaporator
iscale.set_scaling_factor(m.fs.ideal.evaporator.area, 1e-3)
iscale.set_scaling_factor(m.fs.ideal.evaporator.U, 1e-3)
iscale.set_scaling_factor(m.fs.ideal.evaporator.delta_temperature_in, 1e-1)
iscale.set_scaling_factor(m.fs.ideal.evaporator.delta_temperature_out, 1e-1)
iscale.set_scaling_factor(m.fs.ideal.evaporator.lmtd, 1e-1)
iscale.set_scaling_factor(m.fs.ideal.evaporator.heat_transfer, 1e-6)

# Calculate scaling factors
iscale.calculate_scaling_factors(m)

Fix the inputs in the model to solve a square problem (i.e., zero degrees of freedom) for initialization.

In [7]:
m.fs.ideal.evaporator.inlet_feed.flow_mass_phase_comp[0, "Liq", "H2O"].fix(9.65)
m.fs.ideal.evaporator.inlet_feed.flow_mass_phase_comp[0, "Liq", "TDS"].fix(0.35)
m.fs.ideal.evaporator.inlet_feed.temperature[0].fix(50.52 + 273.15)
m.fs.ideal.evaporator.inlet_feed.pressure[0].fix(101325)
m.fs.ideal.evaporator.outlet_brine.temperature[0].fix(60 + 273.15)
m.fs.ideal.evaporator.U.fix(1e3)
m.fs.ideal.evaporator.area.fix(400)
m.fs.ideal.evaporator.delta_temperature_out.fix(5)
m.fs.ideal.evaporator.delta_temperature_in.fix(30)

Initialize the `Evaporator` unit and create a solver object to select `IPOPT` as the NLP solver. We include the `assert` keyword to check that we have a square problem followed by the solution of the initialization model.

In [8]:
m.fs.ideal.evaporator.initialize(outlvl=idaeslog.WARNING)

solver = get_solver('ipopt')
assert degrees_of_freedom(m) == 0

ideal_init_results = solver.solve(m, tee=False)

print("The initialization solver termination status is {}".format(ideal_init_results.solver.termination_condition))

The initialization solver termination status is optimal


To solve the simulation example, we unfix some of the inputs that were fixed during initialization and fix the pressure in the evaporator chamber at the desired value of $30,000 \; \rm Pa$.

In [9]:
m.fs.ideal.evaporator.area.unfix()
m.fs.ideal.evaporator.U.unfix()
m.fs.ideal.evaporator.outlet_brine.temperature[0].unfix()
m.fs.ideal.evaporator.outlet_brine.pressure[0].fix(30e3)

Since the overall heat transfer coefficient $\rm U$ and area $\rm A$ of evaporator are unfixed, we add a Pyomo `Expression` to calculate a a new term $\rm UA$ that represents the product of both variables. To avoid moving from undesirable designs, we also add a lower and an upper bound for $\rm U$ as constraints (reference [2]).

In [10]:
@m.fs.ideal.Expression(doc="Overall heat trasfer coefficient and area term")
def UA_term(b):
    return b.evaporator.area*b.evaporator.U

@m.fs.ideal.Constraint(doc="Overall heat trasfer coefficient lower bound")
def U_lower_bound(b):
    return b.evaporator.U >= 500

@m.fs.ideal.Constraint(doc="Overall heat trasfer coefficient upper bound")
def U_upper_bound(b):
    return b.evaporator.U <= 2500

Add an upper bound to the temperature in the evaporator chamber to avoid damages to the equipment (e.g., extreme fouling). 

In [11]:
@m.fs.ideal.Constraint(doc="Evaporator temperature upper bound")
def temperature_upper_bound(b):
    return b.evaporator.outlet_brine.temperature[0] <= (73 + 273.15)

Solve the ideal evaporator model simulation example.

In [12]:
ideal_results = solver.solve(m, tee=False)
print("The ideal solver termination status is {}".format(ideal_results.solver.termination_condition))

The ideal solver termination status is optimal


Display the evaporator results.

In [13]:
print('Evaporator area (m2): ', pyo.value(m.fs.ideal.evaporator.area))
print('Evaporator heat (MW): ', pyo.value(m.fs.ideal.evaporator.heat_transfer)*1e-6)
print('Water activity coefficient: ', value(m.fs.ideal.act_coeff))

Evaporator area (m2):  831.4456557855501
Evaporator heat (MW):  16.910574337450765
Water activity coefficient:  1


# Part II: Evaporator in Nonideal System

To separate the seawater into freshwater and a concentrated brine under nonideal thermodynamic conditions, the electrolyte NRTL method `ENRTL` from IDAES is used to calculate the activity coefficient $\gamma_i$ of water, as the solvent.

To start the construction of the nonideal evaporator model example, we start by importing the IDAES `ENRTL` model and other properties and models needed by it. Note that, since we use the same Pyomo and IDAES libraries that were used in the ideal evaporator example, there is no need to import them again here.

In [14]:
from idaes.models.properties.modular_properties.eos.enrtl import ENRTL
from idaes.core import AqueousPhase, Solvent, Apparent, Anion, Cation
from idaes.models.properties.modular_properties.eos.enrtl_reference_states import (Symmetric, Unsymmetric)
from idaes.models.properties.modular_properties.base.generic_property import StateIndex
from idaes.models.properties.modular_properties.state_definitions import FpcTP, FTPx
from idaes.models.properties.modular_properties.pure.electrolyte import relative_permittivity_constant

Declare a new block to save the nonideal evaporator unit, variables, and constraints.

In [15]:
m.fs.nonideal = Block()

m.fs.nonideal.evaporator = Evaporator(property_package_feed=m.fs.properties_feed,
                                      property_package_vapor=m.fs.properties_vapor)

Declare a dictionary of configuration arguments called `configuration` that includes parameters and equations needed by the eNRTL method. For this example, we assume a symmetric reference state for eNRTL and the nonrandomness factor $\alpha$ and the binary interaction parameter $\tau$ are obtained from reference [1]. 

In [16]:
class ConstantVolMol:
    def build_parameters(b):
        b.vol_mol_pure = Param(initialize=18e-6, 
                               units=pyunits.m**3 / pyunits.mol, 
                               mutable=True)

    def return_expression(b, cobj, T):
        return cobj.vol_mol_pure

# Configuration dictionary
configuration = {
    "components": {
        "H2O": {
            "type": Solvent,
            "vol_mol_liq_comp": ConstantVolMol,
            "relative_permittivity_liq_comp": relative_permittivity_constant,
            "parameter_data": {
                "mw": (18.01528e-3, pyunits.kg/pyunits.mol),
                "relative_permittivity_liq_comp": 78.54
            }
        },
        "NaCl": {
            "type": Apparent,
            "dissociation_species": {"Na+": 1, "Cl-": 1}
        },
        "Na+": {
            "type": Cation,
            "charge": +1,
            "parameter_data": {
                "mw": (22.990e-3, pyunits.kg/pyunits.mol)
            }
        },
        "Cl-": {
            "type": Anion,
            "charge": -1,
            "parameter_data": {
                "mw": (35.453e-3, pyunits.kg/pyunits.mol)
            }
        }
    },
    "phases": {
        "Liq": {
            "type": AqueousPhase,
            "equation_of_state": ENRTL,
            "equation_of_state_options": {
            "reference_state": Symmetric
            }
        }
    },
    "base_units": {
        "time": pyunits.s,
        "length": pyunits.m,
        "mass": pyunits.kg,
        "amount": pyunits.mol,
        "temperature": pyunits.K
    },
    "state_definition": FpcTP,
    "state_components": StateIndex.true,
    "pressure_ref": 101325,
    "temperature_ref": 298.15,
    "parameter_data": {
        "Liq_tau": {
            # From reference [1]
            ("H2O", "Na+, Cl-"): 8.885,
            ("Na+, Cl-", "H2O"): -4.549
        }
    },
    "default_scaling_factors": {
        ("flow_mol_phase_comp", ("Liq", "Na+")): 1e1,
        ("flow_mol_phase_comp", ("Liq", "Cl-")): 1e1,
        ("flow_mol_phase_comp", ("Liq", "H2O")): 1e-1,
        ("mole_frac_comp", "Na+"): 1e2,
        ("mole_frac_comp", "Cl-"): 1e2,
        ("mole_frac_comp", "H2O"): 1,
        ("mole_frac_phase_comp", ("Liq", "Na+")): 1e2,
        ("mole_frac_phase_comp", ("Liq", "Cl-")): 1e2,
        ("mole_frac_phase_comp", ("Liq", "H2O")): 1,
        ("flow_mol_phase_comp_apparent", ("Liq", "NaCl")): 1e1,
        ("flow_mol_phase_comp_apparent", ("Liq", "H2O")): 1e-1,
        ("mole_frac_phase_comp_apparent", ("Liq", "NaCl")): 1e3,
        ("mole_frac_phase_comp_apparent", ("Liq", "H2O")): 1
    }
}

Create an instance of the `GenericParameterBlock` component and provide the eNRTL configuration dictionary.

In [17]:
m.fs.nonideal.prop_enrtl = GenericParameterBlock(**configuration)

Declare a new block to include the generic properties needed by eNRTL and construct a `StateBlock` associated with it.

In [18]:
m.fs.nonideal.enrtl_state = Block()
m.fs.nonideal.enrtl_state.properties = m.fs.nonideal.prop_enrtl.build_state_block([0])

# Rename eNRTL properties block
sb_enrtl = m.fs.nonideal.enrtl_state.properties[0]

Declare a new function to populate the eNRTL `StateBlock`. Here, we choose to use the component flow, temperature, and pressure as the state variables or `FcTP`. 

In [19]:
def populate_enrtl_state_vars(blk, base="FpcTP"):
    blk.temperature = 298.15
    blk.pressure = 101325

    if base == "FpcTP":
        feed_flow_mass = 10  # kg/s
        feed_mass_frac_comp = {"Na+": 0.013768116,
                               "Cl-": 0.021231884}
        feed_mass_frac_comp["H2O"] = (
            1 - sum(x for x in feed_mass_frac_comp.values())
        )
        mw_comp = {"H2O": 18.015e-3,
                   "Na+": 22.990e-3,
                   "Cl-": 35.453e-3}

        for j in feed_mass_frac_comp:
            blk.flow_mol_phase_comp["Liq", j] = (
                feed_flow_mass * feed_mass_frac_comp[j] / mw_comp[j]
            )
            if j == "H2O":
                blk.flow_mol_phase_comp["Liq", j] /= 2

populate_enrtl_state_vars(sb_enrtl, base="FpcTP")

Since the `ENRTL` IDAES method involves the calculation of properties based on individual solute ions and the seawater property package is in terms of total dissolved solids (TDS), we convert the flow from TDS to the respective flow of ions $\rm Na^+$ and $\rm Cl^-$ in the solute. To make that conversion, we declare the set of ions and their stoichiometric coefficient in the solute molecule and used them to calculate the mass ratio of each ion in the solute molecule. 

In [20]:
m.fs.nonideal.set_ions = Set(initialize=["Na+", "Cl-"])
m.fs.nonideal.ion_coeff = {"Na+": 1, "Cl-": 1}
m.fs.nonideal.enrtl_state.mol_mass_ion_molecule = sum(
    m.fs.nonideal.ion_coeff[j] * sb_enrtl.mw_comp[j]
    for j in m.fs.nonideal.set_ions
)
m.fs.nonideal.enrtl_state.mass_ratio_ion = {
    "Na+": sb_enrtl.mw_comp["Na+"]/m.fs.nonideal.enrtl_state.mol_mass_ion_molecule,
    "Cl-": sb_enrtl.mw_comp["Cl-"]/m.fs.nonideal.enrtl_state.mol_mass_ion_molecule
}

Add constraints to link the temperature, pressure, and mass flowrate of the evaporator brine with the eNRTL properties block. Note that by using the mass ratio calculated above, the flow mass of TDS is converted into the respective ions in the solute $\rm NaCl$.

In [21]:
@m.fs.nonideal.enrtl_state.Constraint()
def eq_enrtl_temperature(b):
    return (
        b.properties[0].temperature == m.fs.nonideal.evaporator.properties_brine[0].temperature
    )

@m.fs.nonideal.enrtl_state.Constraint()
def eq_enrtl_pressure(b):
    return (
        b.properties[0].pressure == m.fs.nonideal.evaporator.properties_brine[0].pressure
    )

@m.fs.nonideal.enrtl_state.Constraint()
def eq_enrtl_flow_mass_H2O(b):
    return (
        b.properties[0].flow_mass_phase_comp["Liq", "H2O"] ==
        m.fs.nonideal.evaporator.properties_brine[0].flow_mass_phase_comp["Liq", "H2O"]
    )

@m.fs.nonideal.enrtl_state.Constraint(m.fs.nonideal.set_ions)
def eq_enrtl_flow_mass_ion_comp(b, j):
    flow_mass_in = b.properties[0].flow_mass_phase_comp["Liq", j]
    return flow_mass_in == (
        (m.fs.nonideal.evaporator.properties_brine[0].flow_mass_phase_comp["Liq", "TDS"]* 
         b.mass_ratio_ion[j])
    )

Deactivate the equality equilibrium constraint from the `Evaporator` unit and replace it with the new equilibrium equation that includes the activity coefficient of water. That is, 

$$ P=x_{\rm H_2O} \; P_{\rm H_2O}^{\rm sat} \; \; \to \; \; P=x_{\rm H_2O} \; \gamma_{\rm H_2O} \; P_{\rm H_2O}^{\rm sat}.$$

The activity coefficient of water $\gamma_{\rm H_2O}$ is calculated using `ENRTL` method and taken from the eNRTL state block.

In [22]:
m.fs.nonideal.evaporator.eq_brine_pressure.deactivate()
@m.fs.nonideal.Constraint(doc="Vapor-liquid equilibrium equation")
def _eq_nonideal_phase_equilibrium(b):
    return b.evaporator.properties_brine[0].pressure == (
        b.enrtl_state.properties[0].act_coeff_phase_comp["Liq", "H2O"]* #nonideal activity coefficient
        b.evaporator.properties_brine[0].mole_frac_phase_comp["Liq", "H2O"]*
        b.evaporator.properties_vapor[0].pressure_sat
    )

Add scaling factors for nonideal evaporator unit.

In [23]:
iscale.set_scaling_factor(m.fs.nonideal.evaporator.area, 1e-3)
iscale.set_scaling_factor(m.fs.nonideal.evaporator.U, 1e-3)
iscale.set_scaling_factor(m.fs.nonideal.evaporator.delta_temperature_in, 1e-1)
iscale.set_scaling_factor(m.fs.nonideal.evaporator.delta_temperature_out, 1e-1)
iscale.set_scaling_factor(m.fs.nonideal.evaporator.lmtd, 1e-1)
iscale.set_scaling_factor(m.fs.nonideal.evaporator.heat_transfer, 1e-6)

# Calculate scaling factors
iscale.calculate_scaling_factors(m)

Fix the inputs in the nonideal evaporator to solve a square problem (i.e., zero degrees of freedom) for initialization. We deactivate the `ideal` block since we are now solving for `nonideal`.

In [24]:
m.fs.ideal.deactivate()

m.fs.nonideal.evaporator.inlet_feed.flow_mass_phase_comp[0, "Liq", "H2O"].fix(9.65)
m.fs.nonideal.evaporator.inlet_feed.flow_mass_phase_comp[0, "Liq", "TDS"].fix(0.35)
m.fs.nonideal.evaporator.inlet_feed.temperature[0].fix(50.52 + 273.15)
m.fs.nonideal.evaporator.inlet_feed.pressure[0].fix(101325)
m.fs.nonideal.evaporator.outlet_brine.temperature[0].fix(60 + 273.15)
m.fs.nonideal.evaporator.U.fix(1e3)
m.fs.nonideal.evaporator.area.fix(400)
m.fs.nonideal.evaporator.delta_temperature_out.fix(5)
m.fs.nonideal.evaporator.delta_temperature_in.fix(30)

Initialize the nonideal `Evaporator` unit. Use `assert` keyword to check that we have a square problem and solve the initialization

In [25]:
m.fs.nonideal.evaporator.initialize(outlvl=idaeslog.WARNING)

assert degrees_of_freedom(m) == 0
nonideal_init_results = solver.solve(m, tee=False)

print(nonideal_init_results.solver.termination_condition)

optimal


To solve the nonideal simulation example, we unfix some of the inputs that were fixed during initialization and fix the pressure in the evaporator chamber.

In [26]:
m.fs.nonideal.evaporator.area.unfix()
m.fs.nonideal.evaporator.U.unfix()
m.fs.nonideal.evaporator.outlet_brine.temperature[0].unfix()
m.fs.nonideal.evaporator.outlet_brine.pressure[0].fix(30e3)

Include expression to calculate term $\rm UA$ that represents the product of the evaporator overall heat transfer coefficient $\rm U$ and its area $\rm A$, add lower and upper bounds for $\rm U$, and a constraint to bound the brine temperature to avoid damages to the equipment.

In [27]:
@m.fs.nonideal.Expression(doc="Overall heat trasfer coefficient and area term")
def UA_term(b):
    return b.evaporator.area*b.evaporator.U

@m.fs.nonideal.Constraint(doc="Overall heat trasfer coefficient lower bound")
def U_lower_bound(b):
    return b.evaporator.U >= 500

@m.fs.nonideal.Constraint(doc="Overall heat trasfer coefficient upper bound")
def U_upper_bound(b):
    return b.evaporator.U <= 2500

@m.fs.nonideal.Constraint(doc="Evaporator chamber temperature upper bound")
def temperature_upper_bound(b):
    return b.evaporator.outlet_brine.temperature[0] <= (73 + 273.15)

Add an expression to calculate the molal concentration of solute $\rm NaCl$ in the brine. This concentration is important since $\tau$ and $\alpha$ parameters in the eNRTL method have a molal concentration limit (for this case, the maximum molality is $6 \; \rm mol \; NaCl/kg \; H_2O$ from reference [1]).

In [28]:
@m.fs.nonideal.Expression(
    doc="Molal concentration of solute in solvent in mol of TDS/kg of H2O")
def molal_conc_solute(b):
    return  (
        (
            b.evaporator.properties_brine[0].flow_mass_phase_comp["Liq", "TDS"]/ 
            m.fs.properties_feed.mw_comp["TDS"] # to convert it to mol/s
        )/b.evaporator.properties_brine[0].flow_mass_phase_comp["Liq", "H2O"]
    )

Solve the nonideal evaporator simulation example.

In [29]:
nonideal_results = solver.solve(m, tee=False)
print("The nonideal solver termination status is {}".format(nonideal_results.solver.termination_condition))

The nonideal solver termination status is optimal


Display the results for nonideal evaporator and print the value of activity coefficient of water and the molal concentration of solute.

In [30]:
print('Evaporator area (m2): ', pyo.value(m.fs.nonideal.evaporator.area))
print('Evaporator heat (MW): ', pyo.value(m.fs.nonideal.evaporator.heat_transfer)*1e-6)
print('Water activity coefficient: ', value(m.fs.nonideal.enrtl_state.properties[0].act_coeff_phase_comp["Liq", "H2O"]))
print('Molal concentration of solute: ', value(m.fs.nonideal.molal_conc_solute))

Evaporator area (m2):  920.7113682047884
Evaporator heat (MW):  18.77080305415105
Water activity coefficient:  0.9833937955900112
Molal concentration of solute:  5.708382722587011
